Load packages & minian

In [2]:
cd "C:/Users/Manip2/SCRIPTS/minian/"

C:\Users\Manip2\SCRIPTS\minian


In [3]:
import os
import numpy as np
from scipy import signal
import quantities as pq
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import sys 
from datetime import datetime
import shutil
from ast import literal_eval
from scipy.signal import find_peaks
from scipy.stats import zscore
from scipy import stats
from itertools import groupby
from IPython.display import display
from scipy.interpolate import griddata
import numpy as np
from scipy.signal import butter, filtfilt, hilbert
import matplotlib.pyplot as plt
from scipy import interpolate
import numpy.matlib
from sklearn.decomposition import PCA
from scipy import stats
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import resample
from scipy.signal import resample_poly
from math import gcd
import warnings
warnings.filterwarnings("ignore")
import sys
class Tee:
    def __init__(self, *files):
        self.files = files
    def write(self, obj):
        for f in self.files:
            f.write(obj)
            f.flush()
    def flush(self):
        for f in self.files:
            f.flush()


minian_path = os.path.join(os.path.abspath('..'),'minian')
print("The folder used for minian procedures is : {}".format(minian_path))
sys.path.append(minian_path)

from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)


The folder used for minian procedures is : C:\Users\Manip2\SCRIPTS\minian


Define path containg cross-registration file

In [6]:
dir= "//10.69.168.1/crnldata/forgetting/Aurelie/MiniscopeOE_data/L2_3_mice/"

Detect the number of kept units in the cross-registration

In [ ]:
data = {}
for dpath in Path(dir).glob('**/PlaceCells_experiment/mappingsAB.pkl'):

    mappfile = open(dpath.parents[0]/ f'mappingsAB.pkl', 'rb')
    mapping = pickle.load(mappfile)
    mapping_sess = mapping['session']   

    mice = dpath.parents[1].parts[-1]
    NeuronType = dpath.parents[2].parts[-1]
    
    print(f"####################################################################################")
    print(f"################################### {mice} ####################################")
    print(f"####################################################################################")

    nb_minian_total=0
    dict_Calcium = {}
    dict_TodropFile = {}

    minian_folders = [f for f in dpath.parents[0].rglob('minian') if f.is_dir()]

    for minianpath in minian_folders: # for each minian folders found in this mouse 

        try: 
            session_path=minianpath.parents[1]        
            tsmini= pd.read_csv(list(session_path.glob('*V4_Miniscope/timeStamps.csv'))[0])['Time Stamp (ms)']
            V4subfolder=False
            session_type=minianpath.parents[2].name
            session_date=minianpath.parents[3].name
            session_time=minianpath.parents[1].name
            session=session_time        
        except:
            session_path=minianpath.parents[2]      
            tsmini= pd.read_csv(list(session_path.glob('*V4_Miniscope/timeStamps.csv'))[0])['Time Stamp (ms)']                      
            V4subfolder=True
            session_type=minianpath.parents[3].name
            session_date=minianpath.parents[4].name
            session_time=minianpath.parents[0].name
            session=session_time
        print(f"Processing {session_type} session: {session} on the {session_date} ")

        minian_ds = open_minian(minianpath)
        dict_Calcium[session] = minian_ds['C'] # calcium traces 
        
        try: 
            try: 
                with open(minianpath.parent / f'TodropFileAB.json', 'r') as f:
                    unit_to_drop = json.load(f)
                    dict_TodropFile[session]  = unit_to_drop
            except: 
                with open(minianpath/ f'TodropFileAB.json', 'r') as f:
                    unit_to_drop = json.load(f)
                    dict_TodropFile[session]  = unit_to_drop
            nb_minian_total+=1
        except:
            print(' !!!! Minian session not validated !!!!')
            continue


        #######################################################################################
        # Distribute Ca2+ intensity & spikes to vigilance states for each sessions #
        #######################################################################################

        # Remove bad units from recordings

        C=dict_Calcium[session]
        unit_to_drop=dict_TodropFile[session]    

        C_sel=C.drop_sel(unit_id=unit_to_drop)

        CalciumSub = pd.DataFrame(C_sel, index=C_sel['unit_id'])

        indexMappList=mapping_sess[session]
        kept_uniq_unit_List=[]
        group_size_List=[]
        mapping['groupsize']=[len(mapping['group'].loc[i].sum()) for i in np.arange(len(mapping['group']))]
        for unit in CalciumSub.index:
            indexMapp = np.where(indexMappList == unit)[0]
            kept_uniq_unit_List.append(str(indexMapp))
            group_size_List.append(mapping['groupsize'].loc[indexMapp].values.tolist()[0])

        nb_unit=len(CalciumSub)
        if nb_unit==0:
            print(f'no cells kept in the session')
            continue  # next iteration
        
        print(f'... {nb_unit}/{len(C)} units kept in the session')

        unique, counts = numpy.unique(group_size_List, return_counts=True)
        print("...", ", ".join(
            f"{counts[i]} units appears in {unique[i]} sessions"
            for i in range(min(len(counts), len(unique)))
        ))

    mapping['group size']=[len(mapping['group'].loc[i].sum()) for i in np.arange(len(mapping['group']))]
    mapping2=mapping.copy()
    mapping2=mapping2[mapping2['group size']>1]
    print(f'{len(mapping2)}/{len(mapping)} units appears in at least 2 sessions (among all detected units : kept and dropped)')

####################################################################################
################################### RL ####################################
####################################################################################
Processing Cheeseboard session: 12_04_32 on the 2025_06_06 
... 25/124 units kept in the session
... 9 units appears in 1 sessions, 9 units appears in 2 sessions, 3 units appears in 3 sessions, 4 units appears in 4 sessions
Processing Cheeseboard session: 12_19_54 on the 2025_06_06 
... 38/121 units kept in the session
... 19 units appears in 1 sessions, 11 units appears in 2 sessions, 3 units appears in 3 sessions, 5 units appears in 4 sessions
Processing 13-53-09_14-08-28 session: 13_53_13 on the SleepAfter 
... 10/75 units kept in the session
... 2 units appears in 1 sessions, 3 units appears in 2 sessions, 1 units appears in 3 sessions, 4 units appears in 4 sessions
Processing 14-24-27_14-40-02 session: 14_24_31 on the SleepAfter 
... 10/63